In [4]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

In [43]:
from tensorflow.keras.models import Sequential,Model,load_model

In [6]:
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

In [8]:
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input

In [49]:
from tensorflow.keras.preprocessing import image

In [9]:
#Resizeing all the images
IMAGE_SIZE=[224,224]
train_path=('/content/drive/My Drive/Deep learning/Datasets/Train')
valid_path=('/content/drive/My Drive/Deep learning/Datasets/Test')

In [10]:
resnet=ResNet50(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)
# Here we import the Resnet50 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

94773248/94765736 [==============================] - 1s 0us/step


In [11]:
#donot train the existing weights
for layer in resnet.layers:
  layer.trainable=False

In [19]:
folders=glob('/content/drive/My Drive/Deep learning/Datasets/Train/*')

In [20]:
folders

['/content/drive/My Drive/Deep learning/Datasets/Train/audi',
 '/content/drive/My Drive/Deep learning/Datasets/Train/mercedes',
 '/content/drive/My Drive/Deep learning/Datasets/Train/lamborghini']

In [21]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [23]:
# our layers - you can add more if you want
x=Flatten()(resnet.output)

In [24]:
len(folders)

3

In [25]:
prediction=Dense(len(folders),activation='softmax')(x)

In [27]:
#Create a model
model=Model(inputs=resnet.input,outputs=prediction)

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [30]:
# tell the model what cost and optimization method to use
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [31]:
# Use the Image Data Generator to import the images from the dataset
training_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [35]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [36]:
## Make sure you provide the same target size as initialied for the image size
training_set=training_datagen.flow_from_directory('/content/drive/My Drive/Deep learning/Datasets/Train',
                                                  target_size=(224,224),
                                                  class_mode='categorical',
                                                  batch_size=32)

Found 64 images belonging to 3 classes.


In [37]:
test_set=test_datagen.flow_from_directory('/content/drive/My Drive/Deep learning/Datasets/Test',
                                          target_size=(224,224),batch_size=32,
                                          class_mode='categorical')

Found 58 images belonging to 3 classes.


In [40]:
# fit the model
# Run the cell. It will take some time to execute
r=model.fit_generator (training_set,validation_data=test_set,
                      epochs=50,
                      steps_per_epoch=len(training_set),
                      validation_steps=len(test_set))

Epoch 1/50
2/2 [==============================] - 49s 24s/step - loss: 7.2885 - accuracy: 0.3438 - val_loss: 6.6510 - val_accuracy: 0.3276
Epoch 2/50
2/2 [==============================] - 1s 414ms/step - loss: 7.4332 - accuracy: 0.2969 - val_loss: 3.5064 - val_accuracy: 0.5172
Epoch 3/50
2/2 [==============================] - 1s 421ms/step - loss: 6.6779 - accuracy: 0.2344 - val_loss: 10.4242 - val_accuracy: 0.1552
Epoch 4/50
2/2 [==============================] - 1s 413ms/step - loss: 6.8084 - accuracy: 0.3438 - val_loss: 3.8735 - val_accuracy: 0.3276
Epoch 5/50
2/2 [==============================] - 1s 420ms/step - loss: 3.3405 - accuracy: 0.4375 - val_loss: 3.5958 - val_accuracy: 0.3276
Epoch 6/50
2/2 [==============================] - 1s 433ms/step - loss: 3.6147 - accuracy: 0.4375 - val_loss: 3.3056 - val_accuracy: 0.5345
Epoch 7/50
2/2 [==============================] - 1s 425ms/step - loss: 4.3205 - accuracy: 0.3906 - val_loss: 1.5026 - val_accuracy: 0.5862
Epoch 8/50
2/2 [====

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [45]:
model.save('model_resnet50.h5')

In [47]:
y_pred=model.predict(test_set)

In [48]:
y_pred=np.argmax(y_pred,axis=1)

In [50]:
model=load_model('model_resnet50.h5')

In [52]:
img=image.load_img('/content/drive/My Drive/Deep learning/Datasets/Test/lamborghini/10.jpg',target_size=(224,224))

In [53]:
x=image.img_to_array(img)
x

array([[[ 17.,   7.,   0.],
        [ 17.,   7.,   0.],
        [ 17.,   7.,   0.],
        ...,
        [ 13.,   2.,   0.],
        [ 14.,   4.,   3.],
        [ 14.,   4.,   3.]],

       [[ 17.,   7.,   0.],
        [ 17.,   7.,   0.],
        [ 18.,   8.,   0.],
        ...,
        [ 14.,   3.,   1.],
        [ 14.,   4.,   3.],
        [ 14.,   4.,   3.]],

       [[ 18.,   8.,   0.],
        [ 18.,   8.,   0.],
        [ 19.,   9.,   0.],
        ...,
        [ 14.,   3.,   1.],
        [ 14.,   4.,   3.],
        [ 14.,   4.,   3.]],

       ...,

       [[209., 129.,  92.],
        [210., 130.,  93.],
        [212., 132.,  95.],
        ...,
        [216., 132.,  96.],
        [214., 129.,  92.],
        [213., 129.,  92.]],

       [[208., 128.,  91.],
        [208., 128.,  91.],
        [211., 131.,  94.],
        ...,
        [215., 131.,  95.],
        [209., 129.,  94.],
        [207., 129.,  93.]],

       [[206., 128.,  90.],
        [206., 128.,  90.],
        [209., 1

In [54]:
x.shape

(224, 224, 3)

In [55]:
x=x/255

In [56]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [58]:
model.predict(img_data)

array([[6.7233341e-04, 7.3178694e-02, 9.2614895e-01]], dtype=float32)

In [59]:

a=np.argmax(model.predict(img_data), axis=1)

In [60]:
a==1

array([False])